# Webサイトへのアクセスとスクレイピング

## Webサイトの生データをPythonで読み込む

In [3]:
# 14・1・1
import requests
url = 'https://tenki.jp/'  # tenki.jpのURL
response = requests.get(url)
response.text[:200]  # 結果の先頭200文字を表示

'<!-- / ## generate at 2025-07-24 10:00:10 -->\n\n\n<!DOCTYPE html>\n<html lang="ja" prefix="og: http://ogp.me/ns#">\n<head>\n<title>日本気象協会 tenki.jp / 天気予報・地震・台風</title>\n<meta charset="UTF-8">\n<meta name="ke'

In [4]:
# 14・1・2
from bs4 import BeautifulSoup
from IPython.display import display, HTML
# soupオブジェクトを得る
soup = BeautifulSoup(response.text, "html.parser")
section = soup.select_one(("body > div#main-column >"
                           "section > div > div#forecast-map"
                           "> a#forecast-map-entry-13101"))
section.text  # 内容を確認

'東京35/260%'

## HTMLを必要な部分だけ理解する

In [5]:
# 14・1・3
a_elem = soup.find(id="forecast-map-entry-13101")
a_elem.text

'東京35/260%'

In [ ]:
# 14・1・4
for elem in soup.find_all(class_="forecast-map-entry"):
    print(elem.text)

## BeautifulSoupについて知る

In [ ]:
# 14・1・5
a_elem = soup.find(id="forecast-map-entry-13101")
a_elem.contents[0]

In [ ]:
# 14・1・6
w_list = []
for elem in soup.find_all(class_="forecast-map-entry"):
    w_dic = {"city": elem.contents[0],
             "weather": elem.find("img").attrs["alt"],
             "max-temp": int(elem.find(class_="max-temp").text),
             "min-temp": int(elem.find(class_="min-temp").text)}
    w_list.append(w_dic)
w_list[0]   # 札幌の天気を表示

## 大量のデータをスクレイピングする

In [ ]:
# 14・1・7
url = 'https://tenki.jp/week/'  # tenki.jpのURL(2週間天気)
response = requests.get(url)  # HTMLを取得
# soupオブジェクトを得る
soup = BeautifulSoup(response.text, "html.parser")
table = soup.find("table", class_="week-thisweek-table")
table # 結果を表示

In [ ]:
# 14・1・8
# 2番目のtrを取得
tr = table.find_all("tr")[1]
tr  # 結果を表示

In [ ]:
# 14・1・8
# 2番目のtrを取得
hokkaido_tr = table.find_all("tr")[1]
hokkaido_tr  # 結果を表示

In [ ]:
# 14・1・9
for td in hokkaido_tr.find_all("td"):
    if "area-name" in td.attrs["class"]:
        # 地域名を取得
        area_name = td.text
        print(area_name)
    else:
        # 天気の情報を取得
        p_list = td.find_all("p")
        span_list = p_list[1].find_all("span")
        weather = p_list[0].text
        max_temp = int(span_list[0].text)
        min_temp = int(span_list[1].text)
        percip = int(p_list[2].text.replace("%", ""))
        print(weather, max_temp, min_temp, percip)

In [ ]:
# 14・1・10
def get_week_weather(tr):
    # trから地域の天気，週間天気を取り出して返す関数
    w_list = []  # 週間天気のリストを初期化
    for td in tr.find_all("td"):
        if "area-name" in td.attrs["class"]:
            # 地域名を取得
            area_name = td.text
        else:
            # 天気の情報を取得
            p_list = td.find_all("p")
            span_list = p_list[1].find_all("span")
            weather = p_list[0].text
            max_temp = int(span_list[0].text)
            min_temp = int(span_list[1].text)
            percip = int(p_list[2].text.replace("%", ""))
            w_dic = {"weather": weather,
                     "max_temp": max_temp,
                     "min_temp": min_temp,
                     "percip": percip}
            w_list.append(w_dic)
            
    return area_name, w_list

In [ ]:
# 14・1・10
get_week_weather(hokkaido_tr)

In [ ]:
# 14・1・11
url = 'https://tenki.jp/week/'  # tenki.jpのURL(2週間天気)
response = requests.get(url)  # HTMLを取得
# soupオブジェクトを得る
soup = BeautifulSoup(response.text, "html.parser")

weathers = []  # 天気の情報用リストを初期化
# 2つのtableをループで取得
for table in soup.find_all("table", class_="week-thisweek-table"):
    # tableの中にあるtrを使ってループ
    for cnt, tr in enumerate(table.find_all("tr")):
        if cnt == 0:  # 最初のtrを飛ばす
            continue
        pname, weather_info = get_week_weather(tr)
        weathers.append((pname, weather_info))
weathers[3][0], weathers[3][1][:3]  # 北陸の情報を確認